In [11]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [5]:
from langchain.tools import Tool

def get_cost_of_living_info(location: str) -> str:
    """Fetches dummy cost of living information for a given location."""
    # Dummy weather data
    cost_of_living = {
        "Berlin" : 15,
        "New-York" : 25,
        "Texas" : 20
    }

    return f"Cost of living in {location}: ${cost_of_living[location]}"

# Initialize the tool
cost_of_living_info_tool = Tool(
    name="get_cost_of_living_info",
    func=get_cost_of_living_info,
    description="Fetches dummy cost of living information for a given location."
)

get_cost_of_living_info(location="Berlin")

'Cost of living in Berlin: $15'

In [6]:
def get_quality_of_living_info(location: str) -> str:
    """Fetches dummy quality of living information for a given location."""
    quality_of_living = {
        "New-York": "Fast-paced, diverse, excellent public transport.",
        "Texas": "Warm weather, growing arts scene, low taxes.",
        "Berlin": "Cold winters, strong cultural scene, affordable housing."
    }

    return f"Quality of living in {location}: {quality_of_living[location]}"

# Initialize the tool
quality_of_living_info_tool = Tool(
    name="get_quality_of_living_info",
    func=get_quality_of_living_info,
    description="Fetches dummy quality of living information for a given location."
)

get_quality_of_living_info(location="Berlin")

'Quality of living in Berlin: Cold winters, strong cultural scene, affordable housing.'

In [7]:
from langchain_groq import ChatGroq

chat = ChatGroq(model="llama3-8b-8192", verbose=True)
tools = [search_tool, cost_of_living_info_tool, quality_of_living_info_tool]
chat_with_tools = chat.bind_tools(tools)

In [8]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

In [9]:
messages = [AIMessage(content="You are an intelligent career decision specialist and have been provided with tools to comapre cost of living and qulity of living for diffrent locations and based on that give user which location they should choose and why."),
    HumanMessage(content="""
I am currently a Data Scientist in Texas, and I have a new offer for a Senior Data Scientist position in Berlin.
Should I take it?
""")]
response = alfred.invoke({"messages": messages})

print("Alfred's Response:")
print(response['messages'][-1].content)

Alfred's Response:
Based on the results, I would say that the cost of living in Berlin ($15) is lower compared to Texas ($20). Additionally, the quality of living in Berlin seems to be quite good with a strong cultural scene and affordable housing, despite having cold winters.

Considering your current role as a Data Scientist in Texas and the offer for a Senior Data Scientist position in Berlin, I would recommend taking the offer in Berlin. The lower cost of living and good quality of living could make a significant difference in your overall quality of life.


In [10]:
response['messages']

[AIMessage(content='You are an intelligent career decision specialist and have been provided with tools to comapre cost of living and qulity of living for diffrent locations and based on that give user which location they should choose and why.', additional_kwargs={}, response_metadata={}, id='9595861c-276f-4566-994d-a8bee3095a6e'),
 HumanMessage(content='\nI am currently a Data Scientist in Texas, and I have a new offer for a Senior Data Scientist position in Berlin.\nShould I take it?\n', additional_kwargs={}, response_metadata={}, id='8f191965-257f-49e5-bde5-fd25ddb6f009'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5ymn', 'function': {'arguments': '{"__arg1":"Berlin"}', 'name': 'get_cost_of_living_info'}, 'type': 'function'}, {'id': 'call_npm4', 'function': {'arguments': '{"__arg1":"Berlin"}', 'name': 'get_quality_of_living_info'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 1248, 'total_tokens': 13